<a href="https://colab.research.google.com/github/benjaminbrown038/NVIDIA/blob/main/notebooks/nvidia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA

- Multi-GPU Training Strategies
- Unoptimized deployment of GPT-J
- Optimizing inference with NVIDIA FasterTransformer library
- Multi-Node Distributed Training Strategies
- GPT_LM_pretrainings_optimizations
- GPT-J deployment with NVIDIA FasterTransformer and Triton Inference server
- Multi-Nodes Distributed Training for Computer Vision
- Mixture of Experts (MoE)
- Sequence Data

## Multi-GPU Training Strategies

In [ ]:
!squeue

In [ ]:
!scancel -u $USER

In [ ]:
!squeue

In [ ]:
!cat /dli/code/pretrain_gpt_1GPU.sh

In [ ]:
%%html
<pre>
   Step 1: Open a terminal session by following the <a href="", data-commandlinker-command="terminal:create-new">Terminal link</a>
   Step 2: Run an interactive session: <font color="green">srun -N 1 --pty /bin/bash</font>
   Step 3: Run the megatron gpt3 pretraining on 1 GPU: <font color="green">bash ./code/pretrain_gpt_1GPU.sh</font>
</pre>

In [ ]:
!squeue

In [ ]:
!sleep 6m
!nvidia-smi

In [ ]:
!grep iteration /dli/megatron/logs/log_1GPU.txt

In [ ]:
!rm -rf /dli/megatron/checkpoints/*

In [ ]:
%%writefile /dli/code/pretrain_gpt_2GPU.sh

In [ ]:
#!/bin/bash

In [ ]:

# Distributed training args
NNODES=1
GPUS_PER_NODE=#FIXEME         # <--- CHANGE HERE
TP_SIZE=1
PP_SIZE=1

# Distributed training
MICRO_BATCH_SIZE=2
GLOBAL_BATCH_SIZE=#FIXEME    # <--- CHANGE HERE

# Model architecture
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
NAME="log_2GPU"

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
            --clip-grad 1.0 \
            --weight-decay 1e-1 \
            --exit-duration-in-mins 1190 \
            "

GPT_ARGS=" \
            --num-layers $NLAYERS \
            --hidden-size $NHIDDEN \
            --num-attention-heads $NHEADS \
            --seq-length $SEQ_LEN \
            --max-position-embeddings $SEQ_LEN \
            --micro-batch-size $MICRO_BATCH_SIZE \
            --global-batch-size $GLOBAL_BATCH_SIZE \
            --train-iters 100 \
            --vocab-file $VOCAB_FILE \
            --merge-file $MERGE_FILE \
            --init-method-std 0.006 \
            $OPTIMIZER_ARGS \
            $EXIT_OPTS \
            "

OUTPUT_ARGS=" \
            --log-interval 10 \
            --save-interval 300 \
            --eval-interval 1000 \
            --eval-iters 10 \
            --tensorboard-dir $TENSORBOARD_PATH \
            --tensorboard-queue-size 1 \
            --log-timers-to-tensorboard \
            --log-batch-size-to-tensorboard \
            --log-validation-ppl-to-tensorboard \
            "
export LAUNCHER="python -u -m torch.distributed.launch \
            --nproc_per_node $GPUS_PER_NODE \
            --nnodes $NNODES \
            --master_addr $MASTER_ADDR \
            --master_port $MASTER_PORT \

export CMD=" \
            /dli/megatron/Megatron-LM/pretrain_gpt.py \
            --tensor-model-parallel-size $TP_SIZE \
            --pipeline-model-parallel-size $PP_SIZE \
            $GPT_ARGS \
            $OUTPUT_ARGS \
            --save $CHECKPOINT_PATH \
            --data-path $DATA_PATH \
            --data-impl mmap \
            --split 949,50,1 \
            --distributed-backend nccl \
            "

bash -c '$LAUNCHER  $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

In [ ]:
%%html
<pre>
   Step 1: Open a terminal session by following the <a href="", data-commandlinker-command="terminal:create-new">Terminal link</a>
   Step 2: Run an interactive session: <font color="green">srun -N 1 --pty /bin/bash</font>
   Step 3: Run the megatron gpt3 pretraining on 1 GPU: <font color="green">bash ./code/pretrain_gpt_2GPU.sh</font>
</pre>

In [ ]:
!squeue

In [ ]:
!nvidia-smi

In [ ]:
!grep iteration /dli/megatron/logs/log_2GPU.txt

In [ ]:
!rm -rf /dli/megatron/checkpoints/*

In [ ]:
!squeue

In [ ]:
!scancel -u $USER

In [ ]:
!squeue

## Unoptimized Deployment of GPT-J

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained("./weights/gpt-j/hf/")
tokenizer = AutoTokenizer.from_pretrained("./weights/gpt-j/hf/")
tokenizer = AutoTokenizer.from_pretrained("./weights/gpt-j/hf/")

In [ ]:
assert torch.cuda.is_available()
device = torch.device("cuda:0")
model.half().to(device)
model = model.eval()

In [ ]:

# Generate a random sentence.
with torch.no_grad():
    output = model.generate(input_ids=None, max_length=128, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

In [ ]:
# Decoding the generated text
for sentence in output:
    sentence = sentence.tolist()
    text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
    print(text)

In [ ]:
input_ids = tokenizer.encode("English: I do not speak French. French: Je ne parle pas français." \
                             "English: See you later! French: À tout à l'heure!" \
                             "English: Where is a good restaurant? French: Où est un bon restaurant?" \
                             "English: What rooms do you have available? French:", return_tensors="pt").cuda(0)

In [ ]:
output = model.generate(input_ids=input_ids, max_length=82, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

In [ ]:
sentence = output[0].tolist()
text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
print(text)

In [ ]:
output = model.generate(input_ids=input_ids, max_length=80, num_return_sequences=1, num_beams=5, temperature=0.7, repetition_penalty=3.0, pad_token_id=tokenizer.eos_token_id)
sentence = output[0].tolist()
text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
print(text)

In [ ]:
input_ids = tokenizer.encode("Create an SQL request to find all users that live in Califorian and have more than 1000 credits.", r)

In [ ]:
output = model.generate(input_ids=input_ids, max_length=82, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

In [ ]:
output = model.generate(input_ids=input_ids, max_length=80, num_return_sequences=1, num_beams=5, temperature=0.7, repetition_penalty=3.0, pad_token_id=tokenizer.eos_token_id)
sentence = output[0].tolist()
text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
print(text)

In [ ]:
import time

execution_time = 0
num_iterations = 10
with torch.no_grad():
    for _ in range(num_iterations):
        start = time.time()
        output = model.generate(input_ids=None, max_length=128, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, eos_token_id=50256)
        end = time.time()
        execution_time += end - start

In [ ]:
print("Average inference time of 128 tokens is:",
     1000 * (execution_time/float(num_iterations)), "ms")

## Optimizing inference with NVIDIA FasterTransformer library

In [ ]:
# Check the SLURM jobs queue
!squeue

In [ ]:
# Cancel admin user jobs
!scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
!squeue

In [ ]:
# import the relevant libraries
import torch
import torchvision
from IPython.display import display_html

def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

# define an image transform
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5),(0.5, 0.5, 0.5))

In [ ]:
# Download the CIFAR10 training dataset
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=2)

In [ ]:
# Download the CIFAR10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)

testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=64,
                                         shuffle=False,
                                         num_workers=2)

In [ ]:
# Show some random training images
import matplotlib.pyplot as plt
import numpy as np

def imshow(images, labels):
    for i in range(8):
        img = images[i] / 2 + 0.5
        npimg = img.numpy()
        plt.subplot(2,4,i+1)
        plt.imshow(np.transpose(npimg, (1, 2 , 0)));
        plt.axis('off');
        plt.title(classes[labels[i]])

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# get some training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
# Show images
imshow(images,labels)

In [ ]:
# Define the CNN
import torch.nn as nn
import torch.nn.functional as F

class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

cnn_net = CNN_Net()

In [ ]:
# Copy the model to GPU 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn_net.to(device)

In [ ]:

# Let's have a look at the Convolutional Neural Network
from torchsummary import summary
summary(cnn_net,input_size=(3,32,32))

In [ ]:
# Define the hyperparameters
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_net.parameters(), lr=0.001,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Tensorboard event recording directory
writer = SummaryWriter('megatron/tensorboard/cifar10')

log_interval=100
batch_size=64
epochs=2

In [ ]:
# Train the CNN
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = cnn_net(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # print the loss and accuracy metrics log_interval mini-batches
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        if i % log_interval == (log_interval - 1):
            print('[epoch %d, iterations %5d] loss: %.3f accuracy: %2f %%' %  (epoch , i + 1, running_loss / log_interval, 100.*correct/total))
            writer.add_scalar("Training Cross Entropy Loss", running_loss / log_interval, i + 1)
            writer.add_scalar("Training Accuracy", 100.*correct/total, i + 1)
            running_loss = 0.0
    # print the last iterations
    print('[epoch %d, iterations %5d] loss: %.3f accuracy: %2f %%' %  (epoch , i + 1, running_loss / ((i % log_interval) + 1), 100.*correct/total))
    writer.add_scalar("Training Cross Entropy Loss", running_loss / ((i % log_interval) + 1), i + 1)
    writer.add_scalar("Training Accuracy", 100.*correct/total, i + 1)

print('Training Done')
writer.add_graph(cnn_net, inputs)
writer.flush()
writer.close()

In [ ]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = cnn_net(images.to(device))
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        c = (predicted == labels.to(device)).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print('Accuracy of the network on the 10000 test images: %2f %%' %
      (100 * correct / total))
for i in range(10):
    print('Accuracy of %5s : %2f %%' %
          (classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
# Define the CNN
import torch.nn as nn
import torch.nn.functional as F

class Net_Parallel(nn.Module):
    def __init__(self):
        super(Net_Parallel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5).to('cuda:0')               # Changed here
        self.pool = nn.MaxPool2d(2, 2).to('cuda:0')                # Changed here
        self.conv2 = nn.Conv2d(6, 16, 5).to('cuda:1')              # Changed here
        self.fc1 = nn.Linear(16 * 5 * 5, 120).to('cuda:1')         # Changed here
        self.fc2 = nn.Linear(120, 84).to('cuda:1')                 # Changed here
        self.fc3 = nn.Linear(84, 10).to('cuda:1')                  # Changed here

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x.to('cuda:0'))))          # Changed here
        x = self.pool(F.relu(self.conv2(x.to('cuda:1'))))          # Changed here
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

cnn_net_pp = Net_Parallel()

In [ ]:
# Define the hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_net_pp.parameters(), lr=0.001,momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Tensorboard event recording directory
writer_pp = SummaryWriter('megatron/tensorboard/cifar10_PP')

log_interval=100
batch_size=64
epochs=2

In [ ]:
%%html

<pre>
   Step 1: Open a terminal session by following the <a href="", data-commandlinker-command="terminal:create-new">Terminal link</a>
   Step 2: Check the GPUs: <font color="green">watch nvidia-smi</font>
</pre

In [ ]:
# Train the CNN with pipeline parallel
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = cnn_net_pp(inputs.to('cuda:0'))                                # Changed here
        loss = criterion(outputs, labels.to('cuda:1'))                           # Changed here
        # Backward pass
        loss.backward()
        optimizer.step()
        torch.cuda.reset_max_memory_allocated(0)
        # print the loss and accuracy metrics log_interval mini-batches
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels.to('cuda:1')).sum().item()                 # Changed here
        if i % log_interval == (log_interval - 1):
            print('[epoch %d, iterations %5d] loss: %.3f accuracy: %2f %%' %  (epoch , i + 1, running_loss / log_interval, 100.*correct/total))
            writer_pp.add_scalar("Training Cross Entropy Loss", running_loss / log_interval, i + 1)
            writer_pp.add_scalar("Training Accuracy", 100.*correct/total, i + 1)
            running_loss = 0.0
    # print the last iterations
    print('[epoch %d, iterations %5d] loss: %.3f accuracy: %2f %%' %  (epoch , i + 1, running_loss / ((i % log_interval) + 1), 100.*correct/total))
    writer.add_scalar("Training Cross Entropy Loss", running_loss / ((i % log_interval) + 1), i + 1)
    writer.add_scalar("Training Accuracy", 100.*correct/total, i + 1)

print('Training Done')
writer_pp.add_graph(cnn_net_pp, inputs)
writer_pp.flush()
writer_pp.close()

In [ ]:
# Have a look at the DeepSpeed config
!cat code/moe/ds_config.json

In [ ]:
# import the relevant library
import deepspeed

# define the argument class, the training arguments, and DeepSpeed
class Args:
    log_interval=100
    batch_size=64
    epochs=2
    deepspeed = True
    deepspeed_config = "code/moe/ds_config.json"
    local_rank= 0

args=Args()

In [ ]:
# define the CNN network
cnn_net_ds = CNN_Net()

# Define the hyperparameters
parameters = filter(lambda p: p.requires_grad, cnn_net_ds.parameters())

# Wrap the CNN network with DeepSpeed
model_engine, optimizer, _, _ = deepspeed.initialize(args=args, model=cnn_net_ds, model_parameters=parameters, training_data=trainset)

# enable mixed precision
fp16 = model_engine.fp16_enabled()

device = model_engine.local_rank
criterion = nn.CrossEntropyLoss()

# Tensorboard event recording directory
writer_ds = SummaryWriter('megatron/tensorboard/cifar10_DS')

In [ ]:
# Train the CNN with DeepSpeed
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        if fp16:
            inputs = inputs.half()
        # zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model_engine(inputs)             # Changed net_cnn to model_engine
        loss = criterion(outputs, labels)

        # Backward pass
        model_engine.backward(loss)                # Changed net_cnn to model_engine
        model_engine.step()                        # Changed net_cnn to model_engine

        # print the loss and accuracy metrics log_interval mini-batches
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        if i % log_interval == (log_interval - 1):
            print('[epoch %d, iterations %5d] loss: %.3f accuracy: %2f %%' %  (epoch , i + 1, running_loss / log_interval, 100.*correct/total))
            writer_ds.add_scalar("Training Cross Entropy Loss", running_loss / log_interval, i + 1)
            writer_ds.add_scalar("Training Accuracy", 100.*correct/total, i + 1)
            running_loss = 0.0

    # print the last iterations
    print('[epoch %d, iterations %5d] loss: %.3f accuracy: %2f %%' %  (epoch , i + 1, running_loss / ((i % log_interval) + 1), 100.*correct/total))
    writer.add_scalar("Training Cross Entropy Loss", running_loss / ((i % log_interval) + 1), i + 1)
    writer.add_scalar("Training Accuracy", 100.*correct/total, i + 1)

print('Training Done')
writer_ds.add_graph(model_engine, inputs)
writer_ds.flush()
writer_ds.close()

In [ ]:
# Kill zombie processes
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
# run the training on 4 GPUs with Data parallel
!deepspeed --num_gpus=4 /dli/code/moe/cifar10_deepspeed.py \
    --deepspeed \
    --deepspeed_config /dli/code/moe/ds_config.json \
    --profile-execution=True \
    --profile-name='zero0'

In [ ]:
%%writefile /dli/code/run_cifar10_deepspeed_2Nodes.sh

In [ ]:
#!/bin/bash

In [ ]:
#SBATCH --job-name=dli_ds


In [ ]:
#SBATCH --nodes=2


In [ ]:
#SBATCH --ntasks-per-node=1


In [ ]:
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)


In [ ]:
#SBATCH -o /dli/megatron/logs/%j.out


In [ ]:
#SBATCH -e /dli/megatron/logs/%j.err

In [ ]:
# Number of nodes
NUM_NODES=2
# Number of GPUs per node
NUM_GPUS=2

deepspeed --num_nodes=${NUM_NODES} --hostfile /dli/code/moe/hostfile --num_gpus=${NUM_GPUS} /dli/code/moe/cifar10_deepspeed.py \
    --deepspeed \
    --deepspeed_config /dli/code/moe/ds_config.json \
    --profile-execution=True \
    --profile-name='zero0_sbatch'

In [ ]:
# Submit the 2 nodes jobs
!sbatch /dli/code/run_cifar10_deepspeed_2Nodes.sh

# Check the SLURM queue
!squeue

In [ ]:
# Check GPU utilization on the master node
!sleep 10
!nvidia-smi

In [ ]:
# Show DeepSpeed config file for Zero stage 3 Offload
!cat /dli/code/moe/ds_config_stage_3.json

In [ ]:
import torch
net = torch.hub.load("pytorch/vision:v0.10.0", "resnet152", force_reload=True, pretrained=True)

In [ ]:
!deepspeed --num_gpus=4 /dli/code/moe/large_model_deepspeed.py \
    --deepspeed \
    --deepspeed_config /dli/code/moe/ds_config_stage_3.json \
    --profile-execution=True \
    --profile-name='zero_resnet152_stage3'

In [ ]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

In [ ]:
# Show DeepSpeed config file for Zero stage 1
!cat /dli/code/moe/ds_config_stage_1.json

In [ ]:
!deepspeed --num_gpus=4 /dli/code/moe/large_model_deepspeed.py \
    --deepspeed \
    --deepspeed_config #FIXEME \
    --profile-execution=True \
    --profile-name=#FIXEME

In [ ]:
!squeue

In [ ]:
# Cancel admin user jobs
!scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
!squeue

## Multi-Node Distributed Training Strategies

## GPT LM Pretrainings Optimizations

## GPT-J deployment with NVIDIA FasterTransformer and Triton Inference server

## Multi-Nodes Distributed Training for Computer Vision

## Mixture of Experts (MoE)

## Sequence Data